# Diving into Pinecone

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
import pinecone
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV'),
)

/opt/homebrew/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
pinecone.info.version()

VersionResponse(server='2.0.11', client='2.2.4')

## Pinecone Indexes

In [4]:
pinecone.list_indexes()

[]

In [8]:
index_name = 'langchain-pinecone'
if index_name not in pinecone.list_indexes():
    print(f'Creating index {index_name}...')
    pinecone.create_index(index_name, dimension=1536, metric='cosine', pods=1)
    print('Done')
else:
    print(f'{index_name} was already created')

Creating index langchain-pinecone...
Done


In [6]:
pinecone.describe_index(index_name)

IndexDescription(name='langchain-pinecone', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

### Deleting an Index

In [7]:
index_name = 'langchain-pinecone'
if index_name in pinecone.list_indexes():
    print(f'Deleting index {index_name}...')
    pinecone.delete_index(index_name)
    print('Done')
else:
    print(f'{index_name} not found')

Deleting index langchain-pinecone...
Done


In [9]:
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Inserting into Pinecone index

In [28]:
import random
vectors = [[random.random() for _ in range(1536)] for _ in range(5)]
ids = list('abcde')

index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

## Updating a vector

In [15]:
index.upsert(vectors=[('b', [0.1] * 1536)])

{'upserted_count': 1}

## Fetching a vector

In [16]:
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.fetch(ids=['a', 'b'])

{'namespace': '',
 'vectors': {'a': {'id': 'a',
                   'metadata': {},
                   'values': [0.861871481,
                              0.0664932,
                              0.0432179235,
                              0.647648752,
                              0.805342257,
                              0.0215914343,
                              0.409598619,
                              0.757602751,
                              0.27027449,
                              0.642814577,
                              0.781431496,
                              0.990382671,
                              0.0887039602,
                              0.908683717,
                              0.216827214,
                              0.34028706,
                              0.0182041265,
                              0.36114049,
                              0.689749181,
                              0.998796225,
                              0.224657342,
               

## Deleting a vector

In [17]:
index.delete(ids=['a', 'b'])

{}

In [18]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 3e-05,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [19]:
index.fetch(ids=['a'])

{'namespace': '', 'vectors': {}}

## Querying

In [30]:
queries = [[random.random() for _ in range(1536)] for _ in range(1)]
index.query(
    vector=queries,
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'c', 'score': 0.751078844, 'values': []},
             {'id': 'e', 'score': 0.750903845, 'values': []},
             {'id': 'a', 'score': 0.750102758, 'values': []}],
 'namespace': ''}